In [ ]:
import numpy as np
import pandas as pd

In [38]:
gcms_list = ["ACCESS_CM2", "ACCESS_ESM1_5", "CanESM5", "CAS_ESM2_0", "CNRM_CM6_1", "EC_Earth3", "EC_Earth3_Veg_LR", "FGOALS_g3", "GFDL_ESM4", "GISS_E2_1_G", "GISS_E2_1_H", "INM_CM4_8", "INM_CM5_0", "IPSL_CM6A_LR", "MIROC6", "MRI_ESM2_0", "NorESM2_LM", "NorESM2_MM"]

future_period = {'NF': pd.date_range(start='2031-01-01', end='2050-12-31', freq='YS'),
                 'MF': pd.date_range(start='2051-01-01', end='2070-12-31', freq='YS'),
                 'FF': pd.date_range(start='2071-01-01', end='2090-12-31', freq='YS')
                 }

scen_list = ["historical", "ssp126", "ssp245", "ssp370", "ssp585"]

In [44]:
element_list = ["pr", "pet", "tas"]
# element = "pet"
for element in element_list:

    # 读取cru数据和gcm数据
    all_cru_series = pd.read_csv("../data/global_mean_series/cru/cru_global_mean_series.txt", sep='\t', parse_dates=['Time'], index_col='Time')
    if element == "tas":
        cru_series = all_cru_series[element].resample('YS').mean()
    else:
        cru_series = all_cru_series[element].resample('YS').sum()
    cru_mean = cru_series['1981-01-01':'2022-12-31'].mean()

    result_df = pd.DataFrame(index=scen_list[1:], columns=future_period.keys())

    for i, scen in enumerate(scen_list[1:]):
        gcm_scen = pd.read_csv(f"../data/global_mean_series/gcm_bc/{element}/{element}_{scen}_global_mean_series.txt", sep='\t', parse_dates=['Time'], index_col='Time')[gcms_list].resample('YS').sum(min_count=1)
        if element == "pet":
            gcm_scen = gcm_scen - 4
        if element == "pr":
            gcm_scen = gcm_scen - 5
        if element == "tas":
            gcm_scen = gcm_scen / 12
            gcm_scen = gcm_scen - 0.3
        for period in future_period.keys():
            period_mean = gcm_scen.loc[future_period[period]].mean()
            if element in ["pr", "pet"]:
                change = (period_mean - cru_mean) / cru_mean * 100
            if element == "tas":
                change = period_mean - cru_mean

            mean_val = change.mean()
            lower = np.percentile(change, 25)
            upper = np.percentile(change, 75)

            if element == "tas":
                cell = f"{mean_val:.2f}°C [{lower:.2f}°C, {upper:.2f}°C]"
            else:
                cell = f"{mean_val:.2f}% [{lower:.2f}%, {upper:.2f}%]"
            result_df.loc[scen, period] = cell
        result_df.to_excel(f"../results/future_climate_change/{element}_future_climate_change_summary.xlsx")

In [45]:
element_list = ["natural_runoff", "actural_evaporation"]
for element in element_list:

    # 读取cru数据和gcm数据
    FNRD_data = np.load(f"../../2025_04_Params_Transplant/Results/GNRD/{element}_GNRD.npy").astype(np.float32)
    FNRD_data[0] = FNRD_data[1]
    FNRD_data[np.isinf(FNRD_data)] = np.nan
    cru_series = pd.Series(np.nanmean(FNRD_data, axis=(1, 2)), index=pd.date_range(start='1901-01-01', end='2022-12-31', freq='MS')).resample('YS').sum()
    cru_mean = cru_series['1981-01-01':'2022-12-31'].mean()

    result_df = pd.DataFrame(index=scen_list[1:], columns=future_period.keys())

    for i, scen in enumerate(scen_list[1:]):
        gcm_scen = pd.read_csv(f"../data/global_mean_series/FNRD/{element}/{element}_{scen}_global_mean_series.txt", sep='\t', parse_dates=['Time'], index_col='Time')[gcms_list].resample('YS').sum(min_count=1)
        if element == "natural_runoff":
            gcm_scen = gcm_scen + 7
            gcm_scen.iloc[0, :] = np.nan
        if element == "actural_evaporation":
            gcm_scen.iloc[0, :] = np.nan
            gcm_scen = gcm_scen - 7
        for period in future_period.keys():
            period_mean = gcm_scen.loc[future_period[period]].mean()
            change = (period_mean - cru_mean) / cru_mean * 100

            mean_val = change.mean()
            lower = np.percentile(change, 25)
            upper = np.percentile(change, 75)
            cell = f"{mean_val:.2f}% [{lower:.2f}%, {upper:.2f}%]"
            result_df.loc[scen, period] = cell
        result_df.to_excel(f"../results/future_climate_change/{element}_future_climate_change_summary.xlsx")